In [1]:
using GLPK
using GLPKMathProgInterface # Loading the GLPK module for using its solver
using JuMP



In [2]:
Mosquitos = [0	0	1	0	0	0	0	0;
            0	0	0	0	0	0	0	0;
            1	0	0	0	0	0	0	1;
            0	0	1	0	0	0	0	0;
            0	0	0	0	0	0	0	0;
            1	0	0	0	0	0	1	0;
            0	1	0	0	0	0	0	0;
            0	0	0	0	0	0	1	0]
Water = [0	0	0	1	0	0	0	0	;
        0	0	0	0	0	0	0	0	;
        0	0	0	0	0	0	0	0	;
        0	1	1	0	0	0	1	0	;
        0	0	0	1	0	0	0	0	;
        1	0	0	0	1	0	1	0	;
        0	0	0	1	1	1	0	0	;
        0	0	0	0	0	0	0	0	]

Wood= [0	0	0	0	0	0	0	1	;
        1	0	0	0	0	0	0	0	;
        0	0	0	0	0	0	0	0	;
        0	0	0	0	1	0	0	0	;
        0	0	1	0	0	0	0	1	;
        0	0	0	0	0	0	1	1	;
        0	0	0	0	0	0	0	0	;
        0	0	0	0	0	0	0	0	]

Swamp = [0	0	0	0	0	0	1	0	;
        0	0	0	1	0	0	0	0	;
        1	0	0	0	0	0	0	0	;
        1	0	1	0	0	0	0	0	;
        0	0	0	0	0	0	1	0	;
        0	0	0	0	0	0	0	1	;
        0	0	0	1	0	1	0	0	;
        0	1	0	0	0	0	0	0	]

8x8 Array{Int64,2}:
 0  0  0  0  0  0  1  0
 0  0  0  1  0  0  0  0
 1  0  0  0  0  0  0  0
 1  0  1  0  0  0  0  0
 0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  1
 0  0  0  1  0  1  0  0
 0  1  0  0  0  0  0  0

In [3]:
elements = ["Water","Wood","Swamp", "Mosquitos"]

4-element Array{ASCIIString,1}:
 "Water"    
 "Wood"     
 "Swamp"    
 "Mosquitos"

In [4]:
CostAt = ["Water"=> 3, "Wood"=> 2,  "Swamp" => -2 , "Mosquitos"=>-3]

Dict{ASCIIString,Int64} with 4 entries:
  "Water"     => 3
  "Swamp"     => -2
  "Wood"      => 2
  "Mosquitos" => -3

In [5]:
CostNear = ["Water"=> 1, "Wood"=> 1, "Swamp" => -2,"Mosquitos"=>-2, ]

Dict{ASCIIString,Int64} with 4 entries:
  "Water"     => 1
  "Swamp"     => -2
  "Wood"      => 1
  "Mosquitos" => -2

In [6]:
#MODEL CONSTRUCTION
#--------------------
m = Model(solver=GLPKSolverLP()) 


#define initial variables
@defVar(m, quality[elements,1:8,1:8] )

@defVar(m, score[1:8,1:8] )

       

# @defVar(m, qWater[1:8,1:8] >= 0)
# @defVar(m, qWood[1:8,1:8] >= 0)
# @defVar(m, qSwamp[1:8,1:8] >= 0)
# @defVar(m, qMosquitos[1:8,1:8] >= 0)

score[i,j] free for all i in {1,2..7,8}, j in {1,2..7,8}

In [7]:
#define quality constraint
for i = 1:8
    for j = 1:8
        for ki in [0,1, -1]
            for kj in [0,1,-1]
                if i + ki >= 1 && i + ki <= 8 && j + kj >= 1 && j+kj <= 8
                    for e in elements
                        if kj == 0 && ki == 0
                            if e == "Water"
                                @addConstraint(m, quality[e,i,j] >= Water[i + ki, j + kj] * CostAt["Water"]) 
                            elseif e == "Wood"
                                @addConstraint(m, quality[e,i,j] >= Wood[i + ki, j + kj] * CostAt["Wood"])
                            elseif e == "Swamp"
                                @addConstraint(m, quality[e,i,j] <= Swamp[i + ki, j + kj] * CostAt["Swamp"])
                            elseif e == "Mosquitos"
                                @addConstraint(m, quality[e,i,j] <= Mosquitos[i + ki, j + kj] * CostAt["Mosquitos"])
                            end
                        else
                            if e == "Water"
                                @addConstraint(m, quality[e,i,j] >= Water[i + ki, j + kj] * CostNear["Water"]) 
                            elseif e == "Wood"
                                @addConstraint(m, quality[e,i,j] >= Wood[i + ki, j + kj] * CostNear["Wood"])
                            elseif e == "Swamp"
                                @addConstraint(m, quality[e,i,j] <= Swamp[i + ki, j + kj] * CostNear["Swamp"])
                            elseif e == "Mosquitos"
                                @addConstraint(m, quality[e,i,j] <= Mosquitos[i + ki, j + kj] * CostNear["Mosquitos"])
                            end
                        end
                    end
                end
            end
        end
    end
end


    

In [8]:

for i in 1:8
    for j in 1:8
        @addConstraint(m, score[i,j] ==  sum{quality[e,i,j] , e in ["Water","Wood"]} 
                                          -sum{quality[e,i,j] , e in ["Swamp","Mosquitos"]}
        )

    end
end
        
        
            

In [9]:
#define objective
@setObjective(m, Min, sum{score[i,j], i=1:8, j=1:8})

score[1,1] + score[1,2] + score[1,3] + score[1,4] + score[1,5] + score[1,6] + score[1,7] + score[1,8] + score[2,1] + score[2,2] + score[2,3] + score[2,4] + score[2,5] + score[2,6] + score[2,7] + score[2,8] + score[3,1] + score[3,2] + score[3,3] + score[3,4] + score[3,5] + score[3,6] + score[3,7] + score[3,8] + score[4,1] + score[4,2] + score[4,3] + score[4,4] + score[4,5] + score[4,6] + score[4,7] + score[4,8] + score[5,1] + score[5,2] + score[5,3] + score[5,4] + score[5,5] + score[5,6] + score[5,7] + score[5,8] + score[6,1] + score[6,2] + score[6,3] + score[6,4] + score[6,5] + score[6,6] + score[6,7] + score[6,8] + score[7,1] + score[7,2] + score[7,3] + score[7,4] + score[7,5] + score[7,6] + score[7,7] + score[7,8] + score[8,1] + score[8,2] + score[8,3] + score[8,4] + score[8,5] + score[8,6] + score[8,7] + score[8,8]

In [10]:
#print(m)

In [11]:
#print(m)

status = solve(m)



:Optimal

In [12]:
# println("Objective value: ", getObjectiveValue(m))
# solution_score= zeros(8,8)
# for i in 1:8
#     for j in 1:8
#         print( getValue(score[i,j]))
#         print("\t")
        
#         solution_score[i,j] = getValue(score[i,j])
#     end
#     println()
# end

#print(indmax(solution_score))
#print(findmax(solution_score))


In [13]:
max_x = 1
max_y = 1
maxval = 0
sumquality = zeros(8,8)
for i in 1:8
    for j in 1:8

        for e in elements
            sumquality[i,j] += getValue(quality[e,i,j])
        end
        print(sumquality[i,j])
        print("\t")
        if maxval <= sumquality[i,j]
            max_x = i
            max_y = j
            maxval = sumquality[i,j]
        end
    end
    println()
end

print("best campsite is at $(max_x), $(max_y)")

1.0	-1.0	-4.0	-1.0	-1.0	-2.0	-1.0	0.0	
-2.0	-3.0	-3.0	-3.0	-1.0	-2.0	-3.0	-3.0	
-3.0	-2.0	-3.0	-2.0	-1.0	2.0	-1.0	-2.0	
-3.0	0.0	-1.0	-2.0	3.0	0.0	0.0	-2.0	
-3.0	-2.0	-1.0	0.0	2.0	-2.0	-2.0	-1.0	
0.0	0.0	-2.0	0.0	1.0	-2.0	0.0	-1.0	
-3.0	-4.0	-3.0	1.0	1.0	0.0	-2.0	-2.0	
-4.0	-4.0	-3.0	-1.0	-1.0	-3.0	-4.0	-2.0	
best campsite is at 4, 5